<a href="https://www.kaggle.com/code/damlasogut/rule-based-classification?scriptVersionId=170483591" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Return on Lead Calculation with Rule Based Classification

* A gaming company wants to create level based personas by using some characteristics of its customers and create segments according to these new customer definitions and estimate how much new customers can earn the company on average according to these segments. For example: It is desired to determine how much a 25-year-old male IOS user from Turkey can earn on average. 
* Persona.csv dataset contains the prices of the products sold by an international gaming company and some demographic information of the users who purchased these products. The dataset consists of records that occur in each sales transaction. This means that the table is not deduplicated. 

* Price: Customer's expenditure amount
* Source: Type of device to which the customer is connected
* Sex: Gender of the customer
* Country: Country of the customer
* Age: Age of the customer

# Question 1: Read the file persona.csv and show general information about the data set.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option("display.max_rows", None)
pd.set_option("display.expand_frame_repr",False)
pd.set_option("display.width", None)
df = pd.read_csv('/kaggle/input/persona/persona.csv')
df_copy=df.copy()
df.head()

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB


In [3]:
df.columns

Index(['PRICE', 'SOURCE', 'SEX', 'COUNTRY', 'AGE'], dtype='object')

In [4]:
df.duplicated().sum() 

3616

In [5]:
df["SOURCE"].unique() 

array(['android', 'ios'], dtype=object)

In [6]:
df["SOURCE"].nunique() 

2

In [7]:
# How many unique PRICE are there?
df["PRICE"].nunique()

6

In [8]:
# How many sales were realised at which PRICE?
df["PRICE"].value_counts()

PRICE
29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: count, dtype: int64

In [9]:
#How many sales were there from which country?
df.groupby("COUNTRY")["PRICE"].count()

COUNTRY
bra    1496
can     230
deu     455
fra     303
tur     451
usa    2065
Name: PRICE, dtype: int64

In [10]:
#What is the total amount earned from sales by country?
df.groupby("COUNTRY").agg({"PRICE": "sum"})

,PRICE
COUNTRY,
bra,51354
can,7730
deu,15485
fra,10177
tur,15689
usa,70225


In [11]:
#What is the number of sales according to SOURCE types?
df.groupby("SOURCE").agg({"PRICE" : "count"})

,PRICE
SOURCE,
android,2974
ios,2026


In [12]:
#What are the PRICE averages by country?
df.groupby(by=['COUNTRY']).agg({"PRICE": "mean"})

,PRICE
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


In [13]:
#What is the average PRICE by SOURCE?
df.groupby(by=['SOURCE']).agg({"PRICE": "mean"})

,PRICE
SOURCE,
android,34.174849
ios,34.069102


In [14]:
#What are the PRICE averages in the COUNTRY-SOURCE breakdown?
df.groupby(by=["COUNTRY", 'SOURCE']).agg({"PRICE": "mean"})

PRICE
COUNTRY SOURCE            
bra     android  34.387029
        ios      34.222222
can     android  33.330709
        ios      33.951456
deu     android  33.869888
        ios      34.268817
fra     android  34.312500
        ios      32.776224
tur     android  36.229437
        ios      33.272727
usa     android  33.760357
        ios      34.371703

* TASK 2: What are the average earnings by COUNTRY, SOURCE, SEX, AGE?

In [15]:
df.groupby(["COUNTRY", 'SOURCE', "SEX", "AGE"]).agg({"PRICE": "mean"}). head(50)

PRICE
COUNTRY SOURCE  SEX    AGE           
bra     android female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897
                       20   31.000000
                       21   37.292683
                       22   34.945946
                       23   31.941176
                       24   36.500000
                       25   35.666667
                       26   33.166667
                       27   26.142857
                       28   22.333333
                       29   37.571429
                       30   45.666667
                       31   35.097561
                       32   39.000000
                       34   24.000000
                       37   33.736842
                       38   34.555556
                       39   43.000000
                       41   32.333333
                       43   44.000000
                       45   33.615385
                       56   37.000000
                male   15   39.344828
                       16   29.666667
                       17   37.157895
                       18   33.054054
                       19   35.216216
                       20   35.400000
                       21   35.250000
                       22   31.000000
                       23   21.500000
                       24   40.428571
                       25   30.428571
                       26   37.095238
                       27   32.636364
                       28   28.230769
                       29   29.714286
                       30   35.363636
                       35   31.500000
                       36   31.857143
                       40   39.625000
                       41   35.666667
                       44   36.500000
                       46   59.000000
                       53   29.000000
        ios     female 15   36.777778

* TASK 3: Sort the output according to PRICE.

In [16]:
agg_df = df.groupby(by=["COUNTRY", 'SOURCE', "SEX", "AGE"]).agg({"PRICE": "mean"}).sort_values(by="PRICE", ascending=False)
agg_df.head()

,,,,PRICE
COUNTRY,SOURCE,SEX,AGE,
bra,android,male,46,59.0
usa,android,male,36,59.0
fra,android,female,24,59.0
usa,ios,male,32,54.0
deu,android,female,36,49.0


* * Convert the names in the index to variable names.

In [17]:
agg_df = agg_df.reset_index() 

* TASK 5: Convert the variable AGE into a categorical variable and add it to agg_df.

In [18]:
age_bins= [0, 18 ,23 ,30 , 40, 66]
labels=['0-18','19-23','24-30','31-40','41-66']
agg_df["AGE_CAT"]= pd.cut(agg_df["AGE"],bins=age_bins,labels=labels,right=True) 
agg_df["AGE_CAT"].head()

0    41-66
1    31-40
2    24-30
3    31-40
4    31-40
Name: AGE_CAT, dtype: category
Categories (5, object): ['0-18' < '19-23' < '24-30' < '31-40' < '41-66']

*  TASK 6: Define new level based customers and add them as variables to the data set.

In [19]:
agg_df["customers_level_based"] = ['_'.join(i).upper() for i in agg_df.drop(["AGE", "PRICE"], axis=1).values]

agg_df.head()


,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,customers_level_based
0,bra,android,male,46,59.0,41-66,BRA_ANDROID_MALE_41-66
1,usa,android,male,36,59.0,31-40,USA_ANDROID_MALE_31-40
2,fra,android,female,24,59.0,24-30,FRA_ANDROID_FEMALE_24-30
3,usa,ios,male,32,54.0,31-40,USA_IOS_MALE_31-40
4,deu,android,female,36,49.0,31-40,DEU_ANDROID_FEMALE_31-40


In [20]:
for row in agg_df.values:
    print(row)

['bra' 'android' 'male' 46 59.0 '41-66' 'BRA_ANDROID_MALE_41-66']
['usa' 'android' 'male' 36 59.0 '31-40' 'USA_ANDROID_MALE_31-40']
['fra' 'android' 'female' 24 59.0 '24-30' 'FRA_ANDROID_FEMALE_24-30']
['usa' 'ios' 'male' 32 54.0 '31-40' 'USA_IOS_MALE_31-40']
['deu' 'android' 'female' 36 49.0 '31-40' 'DEU_ANDROID_FEMALE_31-40']
['deu' 'android' 'female' 51 49.0 '41-66' 'DEU_ANDROID_FEMALE_41-66']
['deu' 'ios' 'male' 20 49.0 '19-23' 'DEU_IOS_MALE_19-23']
['usa' 'android' 'female' 47 49.0 '41-66' 'USA_ANDROID_FEMALE_41-66']
['fra' 'ios' 'male' 20 49.0 '19-23' 'FRA_IOS_MALE_19-23']
['usa' 'ios' 'male' 59 46.5 '41-66' 'USA_IOS_MALE_41-66']
['deu' 'android' 'male' 15 46.5 '0-18' 'DEU_ANDROID_MALE_0-18']
['bra' 'android' 'female' 30 45.666666666666664 '24-30'
 'BRA_ANDROID_FEMALE_24-30']
['usa' 'android' 'female' 61 45.666666666666664 '41-66'
 'USA_ANDROID_FEMALE_41-66']
['tur' 'ios' 'male' 24 45.0 '24-30' 'TUR_IOS_MALE_24-30']
['deu' 'android' 'male' 34 45.0 '31-40' 'DEU_ANDROID_MALE_31-40'

In [21]:
agg_df["customers_level_based"] = [row[0].upper() + "_" + row[1].upper() + "_" + row[2].upper() + "_" + row[5].upper() for row in agg_df.values]
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,customers_level_based
0,bra,android,male,46,59.0,41-66,BRA_ANDROID_MALE_41-66
1,usa,android,male,36,59.0,31-40,USA_ANDROID_MALE_31-40
2,fra,android,female,24,59.0,24-30,FRA_ANDROID_FEMALE_24-30
3,usa,ios,male,32,54.0,31-40,USA_IOS_MALE_31-40
4,deu,android,female,36,49.0,31-40,DEU_ANDROID_FEMALE_31-40


In [22]:
agg_df = agg_df[["customers_level_based", "PRICE"]]
agg_df.head()


,customers_level_based,PRICE
0,BRA_ANDROID_MALE_41-66,59.0
1,USA_ANDROID_MALE_31-40,59.0
2,FRA_ANDROID_FEMALE_24-30,59.0
3,USA_IOS_MALE_31-40,54.0
4,DEU_ANDROID_FEMALE_31-40,49.0


In [23]:
for i in agg_df["customers_level_based"].values:
    print(i.split("_"))

['BRA', 'ANDROID', 'MALE', '41-66']
['USA', 'ANDROID', 'MALE', '31-40']
['FRA', 'ANDROID', 'FEMALE', '24-30']
['USA', 'IOS', 'MALE', '31-40']
['DEU', 'ANDROID', 'FEMALE', '31-40']
['DEU', 'ANDROID', 'FEMALE', '41-66']
['DEU', 'IOS', 'MALE', '19-23']
['USA', 'ANDROID', 'FEMALE', '41-66']
['FRA', 'IOS', 'MALE', '19-23']
['USA', 'IOS', 'MALE', '41-66']
['DEU', 'ANDROID', 'MALE', '0-18']
['BRA', 'ANDROID', 'FEMALE', '24-30']
['USA', 'ANDROID', 'FEMALE', '41-66']
['TUR', 'IOS', 'MALE', '24-30']
['DEU', 'ANDROID', 'MALE', '31-40']
['TUR', 'IOS', 'FEMALE', '0-18']
['BRA', 'ANDROID', 'FEMALE', '41-66']
['BRA', 'IOS', 'FEMALE', '19-23']
['TUR', 'ANDROID', 'FEMALE', '31-40']
['BRA', 'ANDROID', 'FEMALE', '31-40']
['TUR', 'IOS', 'MALE', '31-40']
['TUR', 'ANDROID', 'MALE', '0-18']
['USA', 'IOS', 'MALE', '24-30']
['BRA', 'IOS', 'MALE', '41-66']
['DEU', 'ANDROID', 'MALE', '19-23']
['USA', 'IOS', 'MALE', '24-30']
['TUR', 'ANDROID', 'FEMALE', '0-18']
['USA', 'ANDROID', 'MALE', '41-66']
['TUR', 'ANDROID

In [24]:
agg_df = agg_df.groupby("customers_level_based").agg({"PRICE": "mean"})

In [25]:
agg_df = agg_df.reset_index()
agg_df.head()

,customers_level_based,PRICE
0,BRA_ANDROID_FEMALE_0-18,35.645303
1,BRA_ANDROID_FEMALE_19-23,34.077340
2,BRA_ANDROID_FEMALE_24-30,33.863946
3,BRA_ANDROID_FEMALE_31-40,34.898326
4,BRA_ANDROID_FEMALE_41-66,36.737179


In [26]:
agg_df["customers_level_based"].value_counts()
agg_df.head()

,customers_level_based,PRICE
0,BRA_ANDROID_FEMALE_0-18,35.645303
1,BRA_ANDROID_FEMALE_19-23,34.077340
2,BRA_ANDROID_FEMALE_24-30,33.863946
3,BRA_ANDROID_FEMALE_31-40,34.898326
4,BRA_ANDROID_FEMALE_41-66,36.737179


In [27]:
agg_df['SEGMENT']=pd.qcut(agg_df['PRICE'],4,labels=["D","C","B","A"])

In [28]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

agg_df.groupby('SEGMENT').agg({'PRICE':['mean','max','sum']})

PRICE                        
              mean        max          sum
SEGMENT                                   
D        29.206780  32.333333   817.789833
C        33.509674  34.077340   904.761209
B        34.999645  36.000000   944.990411
A        38.691234  45.428571  1044.663328

* TASK 7: Segment new customers (USA_ANDROID_MALE_0_18).

In [29]:
agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4, labels=["D", "C", "B", "A"])
agg_df.head(30)

,customers_level_based,PRICE,SEGMENT
0,BRA_ANDROID_FEMALE_0-18,35.645303,B
1,BRA_ANDROID_FEMALE_19-23,34.077340,C
2,BRA_ANDROID_FEMALE_24-30,33.863946,C
3,BRA_ANDROID_FEMALE_31-40,34.898326,B
4,BRA_ANDROID_FEMALE_41-66,36.737179,A
5,BRA_ANDROID_MALE_0-18,34.805861,B
6,BRA_ANDROID_MALE_19-23,31.673243,D
7,BRA_ANDROID_MALE_24-30,33.413919,C
8,BRA_ANDROID_MALE_31-40,34.327381,B
9,BRA_ANDROID_MALE_41-66,40.041667,A


In [30]:
agg_df.groupby("SEGMENT").agg({"PRICE": "mean"})

,PRICE
SEGMENT,
D,29.206780
C,33.509674
B,34.999645
A,38.691234
